# Cuantitative comparision of same condition in different studies

The goal of this notebook is to implement an automated comparision between conditions of different studies that:
- Checks for the consistency of predictions
- Accounts for study-level variability
- Infer core functions and genes/reactions that are condition-specific